# Geoadmin migration path

## Current Issues
The current `mf-geoadmin3` app faces some issues that need to be addressed in the (near) future:
- end of life: angularJS (aka angular 1.X) is in LTS maintenance mode and will be discontinued in mid-2021
- slow: the current app is slow (see e.g. [performance analysis done for MVT](https://ltboc.infra.bgdi.ch/notebooks/mvt/ol-mapbox-performance-comparison.html) which shows that a considerable amount of time is lost during startup of the app)
- complex: despite the modularization, the current app is fairly complex (see dependency analysis below) and difficult to extend and maintain for non-angularJS experts

## Goals for next-gen app
The next-generation SPA should meet these goals:
- as much framework-agnostic code as possible: the new javascript standard ES6 introduced the concept of *modules*, allowing to natively modularize JS code without the need to use a framework (which was the case in the days when angularJS was invented)
- code documentation: the current code is hardly commented and documented, this should be improved
- follow the [Flux](https://facebook.github.io/flux/docs/in-depth-overview.html#content) principle: separate *state*, *logic* and *ui*
- modern build system: webpack is the de-facto standard (or at least very widely used) used to build (compile, minify) *module*-arized JS-libs
- local development: it should be possible to develop and serve the app locally (which comes for free when using webpack with webpack-dev-server)
- PWA: with choosing the correct JS frameworks, PWA's and/or native apps come almost for free (e.g. ionic, vue-native, ...)

## Development approach
In principle, there exist two fundamentally different ways of how to approach development, either green-field approach or a transformation approach, both with risks and advantages. The first question that needs to be answered for both of them is whether the feature set should remain exactly the same or whether some of the existing features can be discontinued. The second question is whether development should happen in an agile manner controlled by us or as a task that is given to an external company and we'd get the final product in the end. 

### Green-field approach
A concise spec does not exist at the moment, however there's the existing code and a test-book that cover many of the use cases. A green-field approach would offer the possibility (if done in an agile way) to start with the most simple features (e.g. show one basemap) and gradually adding features existing in the current app, thereby deciding whether or not to keep this feature. The risks with the green-field approach are that 
- the necessary effort is hard to estimate
- we would have to 'switch' at one point
- we would have to maintain the 'old' app while developing the new one
- we would potentially have to add features to the old app and the new one -> moving target

### Transformation approach
The biggest advantage of the transformation approach is that there's always a production-ready version of the app available. The transformation approach would implicitly require a careful analysis of the existing code, minimizing the risk of 'hidden' business logic being lost. Decisions on whether or not to keep and migrate features could be done incrementally. The disadvantage is that some extra work and code overhead might be required due to the transformation.

## Dependency analysis
In order to find possible leaf nodes that could serve as a start for the migration and in general to understand the dependencies of the individual modules better, a dependency analysis has been made that is (partly) visualized in the following graph.
![geoadmin dependency graph](assets/graph.svg)

# Project Mithridate
*Project Mithridate* is a proof of concept to demonstrate the feasibility of an 'in-app' migration path, gradually migrating parts of the code from the old to the new app. The basic idea is to use a wrapper (`ngVue` in the POC) to expose code in the 'new' app as angularJS modules that can be used in the old app.

The code organisation looks as follows:
```mf-geoadmin:
src                                                 # the 'old' angularJS code
    |_components
    |_js
    |_lib
    |_...
vueApp
    |_dist                                          # contains the build libs, ready to be included in the 'old' angularjs app
        |_css
        |   |_appVueLib.css
        |_js
            |_appVueLib_NgVueBridge.js
            |_appVueLib_VendorsDependencies.js
            |_appVueLib.js
    |_src                                           # contains the 'new' app code
        |_DEV                                       # helper stuff needed only for development
        |   |_index.vue.js
        |_js                                        # contains framework-agnostic ES6 js-module code
        |   |_utils
        |       |_urlUtils.js
        |       |_...
        |_ngVueBridgeCode                           # contains code that only exists during migration phase
        |   |_ngVueComponentsModule.js
        |   |_ngVueDirectives.js
        |   |_ngVueServices.js
        |_vue                                       # contains the directly view-related components
            |_components
            |   |_ZoomButton.vue
            |_App.vue                               # the entry-point of the 'new' app, once all functionality is migrated
    |_package.json                                  # new app has its own package.json
    |_webpack.config.js                             # new app is built using webpack (including local dev-server)
```

Code than can be framework agnostic should be framework agnostic and should go into `vueApp/src/js`. `vue` should only contain parts that are directly related to UI and contain as little business logic as possible.

Once the migration has finished and no code remains in the 'old' app, the `DEV` and `ngVueBridgeCode` can be removed as well as everything outside of `vueApp`.

A working POC can be inspected under XYZ where
- an existing service has been migrated to the new app (`gaGeomUtils`, now in `src/js/ol/geometryUtils.js` with bridge code in `src/ngVueBridgeCode/ngVueServices.js`)

## Todo
Things that remain to be done:

- make sure production build works
- check performance / increased size of build
- decide on whether exact feature parity is required, think about new features that could be realized easier with the right code architecture 
- decide on which ui framework to use (strong favour for vue.js for its simplicity and size)
- decide on which test framework to use for new app